In [0]:
import numpy as np

### Размеры матриц

Пусть:
1. Матрица X имеет размерность (m, n), где m - количество признаков, а n - количество данных
2. Матрица Y имеет размерность (1, n)
3. Матрица весов имеет размерность (m, 1)
4. Смещение считаем отдельно



In [0]:
def log_loss(a, y):
    if y == 1:
        return -y * np.log(a)
    return -(1 - y) * np.log(1 - a)


def log_loss_vect(a, y):
    y_hat = np.squeeze(a).copy()
    y = np.squeeze(y)

    pos_inds = np.where(y == 1)
    neg_inds = np.where(y == 0)

    y_hat[pos_inds] = - np.log(y_hat[pos_inds])
    y_hat[neg_inds] = - np.log(1 - y_hat[neg_inds])
    return y_hat


def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [0]:
lr = 0.0001

def forward_pass(x, w, b):
    """
    Функция делает прямой проход
    :params: w - матрица весов  (k, 1)
             x - матрица данных (k, n)
             b - скаляр
    :return: z = WX + b - вектор (1, n)
             a = sigmoid(z)
    """
    # forward pass
    z = w.T @ x + b
    a = sigmoid(z)
    return z, a

def backward_pass(z, a, x):
    """
    Функция делает обратный проход
    :params: z - вектор (1, n)
             a - вектор (1, n)
             x - матрица данных (k, n)
    :return: dw = dL/dw - вектор (k, 1)
             db = dl/db - скаляр
    """
    # backward pass
    dz, dw, db = 0, 0, 0
    dz = a - y
    dw = (dz @ x.T).sum(axis=0)
    db = dz.sum()
    return dw, db

def gradient_step(w, b, lr, dw, db):
    """
    Функция обновляет веса
    :params: lr - learning rate
             dw = dL/dw - вектор (k, 1)
             db = dl/db - скаляр
             w - вектор весов (k, 1)
             b - скаляр смещения
    :return: w - вектор весов (k, 1)
             b - скаляр смещения
    """
    # gradient update
    w -= lr * dw.reshape(-1, 1)
    b -= lr * db
    return w, b

def train_iteration(x, y, w, b, lr=lr):
    z, a = forward_pass(x, w, b)
    dw, db = backward_pass(z, a, x)
    w, b = gradient_step(w, b, lr, dw, db)
    return w, b


In [252]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

# загрузка данных для бинарной классификации
x = data['data'].T
y = data['target'].T
x = (x - np.mean(x)) / np.std(x)

# Инициализация весов + гиперпараметры
w = np.zeros((x.shape[0], 1))
b = 0
lr = 0.001

# тренировочные итерации
epochs = 100
for i in range(epochs):

    # обучение
    w, b = train_iteration(x, y, w, b, lr=lr)

    # отображение статистик
    _, a = forward_pass(x, w, b)
    if i % 20 == 0:
        print('Eepoch: {}/{} | Log-Loss: {:.3f}'.format(i, epochs, log_loss_vect(a, y).mean()))


Eepoch: 0/100 | Log-Loss: 0.700
Eepoch: 20/100 | Log-Loss: 0.284
Eepoch: 40/100 | Log-Loss: 0.247
Eepoch: 60/100 | Log-Loss: 0.230
Eepoch: 80/100 | Log-Loss: 0.219
